In [1]:
import numpy as p
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW, get_scheduler
from datasets import load_metric, list_metrics

In [2]:
train = pd.read_csv('/kaggle/input/clickbait-task2/train_data.csv')
val = pd.read_csv('/kaggle/input/clickbait-task2/val_data.csv')

In [3]:
train.head()

,text,output,labels
0,['wes welker wanted dinner with tom brady but ...,['how about that morning we go throw'],['passage']
1,['nasa sets date for full recovery of ozone ho...,['2070'],['phrase']
2,['this is what makes employees happy and its ...,['intellectual stimulation'],['phrase']
3,['passion is overrated \xa07 work habits you n...,['purpose connects us to something bigger and ...,['multi']
4,['the perfect way to cook rice so that its per...,['in a rice cooker'],['phrase']


In [4]:
val.head()

,text,output,labels
0,['five nights at freddys sequel delayed for we...,['some of the plot elements are so disturbing ...,['passage']
1,['why arizona sheriff joe arpaios fate could h...,"['intentionally', 'could transform a court cas...",['multi']
2,['heres how much you should be tipping your ha...,['20'],['phrase']
3,"['harry potter alums reunite for new movie', '...","['alan rickman rupert grint', 'cbgb']",['multi']
4,['a man swallowed a microsd card and you wont ...,['a man who swallowed a 64gb microsd card and ...,['passage']


In [5]:
train_inputs = train['text'].astype(str).tolist()
train_outputs = train['output'].astype(str).tolist()

In [6]:
len(train_inputs)

3200

In [7]:
val_inputs = val['text'].astype(str).tolist()
val_outputs = val['output'].astype(str).tolist()

In [8]:
model_name = 'facebook/bart-base'
tokenizer = BartTokenizer.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [9]:
train_encodings = tokenizer(train_inputs, truncation=True, padding=True, max_length=512)
train_labels = tokenizer(train_outputs, truncation=True, padding=True, max_length=128)

val_encodings = tokenizer(val_inputs, truncation=True, padding=True, max_length=512)
val_labels = tokenizer(val_outputs, truncation=True, padding=True, max_length=128)


In [10]:
print(len(train_inputs), len(train_outputs))
print(len(val_inputs), len(val_outputs))

3200 3200
400 400


In [11]:
class CustomDataset(Dataset):
  def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

  def __len__(self):
        return len(self.encodings['input_ids'])

  def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

In [12]:
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [14]:
print(len(train_encodings['input_ids']), len(train_labels['input_ids']))
print(len(val_encodings['input_ids']), len(val_labels['input_ids']))

3200 3200
400 400


In [15]:
lr = 5e-5

num_epochs = 10

model = BartForConditionalGeneration.from_pretrained(model_name)

optimizer = AdamW(model.parameters(), lr = lr)

num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
print('Using device:', device)

Using device: cuda


In [17]:
pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.3 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Model Trainng

In [19]:
for epoch in range(num_epochs):
  model.train()
  for batch in train_loader:
        
    batch = {k: v.to(device) for k, v in batch.items()}
    
    optimizer.zero_grad()

    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()

  print(f"Epoch: {epoch +1} / {num_epochs}, Loss: {loss.item()}")

  model.eval()
  predictions = []
  references = []

  for batch in val_loader:
      batch = {k: v.to(device) for k, v in batch.items()}
      input_ids = batch['input_ids']
      inputs = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
      outputs = model.generate(input_ids)
      preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
      labels = batch['labels']
      refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
      predictions.extend(preds)
      references.extend(refs)
        
  tokenized_predictions = [word_tokenize(pred) for pred in predictions]
  tokenized_references = [word_tokenize(ref) for ref in references]

  meteor_scores = [meteor_score([ref], pred) for pred, ref in zip(tokenized_predictions, tokenized_references)]
  average_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0
  print(f"METEOR Score: {average_meteor}")

Epoch: 1 / 10, Loss: 0.6868572235107422


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


METEOR Score: 0.399003051092812
Epoch: 2 / 10, Loss: 0.23143444955348969
METEOR Score: 0.39330490527252043
Epoch: 3 / 10, Loss: 0.06341668963432312
METEOR Score: 0.3937230942646888
Epoch: 4 / 10, Loss: 0.0632033497095108
METEOR Score: 0.42191222924312194
Epoch: 5 / 10, Loss: 0.07087889313697815
METEOR Score: 0.42399955376777526
Epoch: 6 / 10, Loss: 0.12990950047969818
METEOR Score: 0.426519771342458
Epoch: 7 / 10, Loss: 0.026988642290234566
METEOR Score: 0.4294346619542744
Epoch: 8 / 10, Loss: 0.052918240427970886
METEOR Score: 0.4389933200298175
Epoch: 9 / 10, Loss: 0.02559727244079113
METEOR Score: 0.4422966434365293
Epoch: 10 / 10, Loss: 0.03247252479195595
METEOR Score: 0.44790504675217835


In [20]:
  test = pd.read_csv('/kaggle/input/clickbait-task2/test.csv')

In [21]:
test.head()

,id,features
0,0,['he tackles a nurse at the hospital then you ...
1,1,"['why you should be selfish at work', 'were al..."
2,2,['the one strange trick that will make you liv...
3,3,['nerd wins scrabble championship with word yo...
4,4,['the bizarre new way to eat eggs that has eve...


In [22]:
test_inputs = test['features'].astype(str).tolist()

In [23]:
test_encodings = tokenizer(test_inputs, truncation=True, padding=True, max_length=512)

In [26]:
class TestDataset(Dataset):
  def __init__(self, encodings):
        self.encodings = encodings

  def __len__(self):
        return len(self.encodings['input_ids'])

  def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

In [28]:
test_data = TestDataset(test_encodings)
test_loader = DataLoader(test_data, batch_size= 8 , shuffle = False)

In [29]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        input_ids = batch['input_ids']
        outputs = model.generate(input_ids)
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(preds)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
clean_predictions = [pred.replace("[", "").replace("]", "").replace("'", "").strip() for pred in predictions]

In [34]:
pred_df = pd.DataFrame({
    'id': test['id'],
    'spoiler': clean_predictions
})

In [35]:
pred_df.head()

,id,spoiler
0,0,['gave him a kidney']
1,1,"['1 prioritise say yes when it matters most',..."
2,2,['meditation']
3,3,['braconid']
4,4,['cured egg yolks are deliciousbut strong beca...


In [36]:
pred_df.to_csv('/kaggle/working/output_BART_lr5e-5_file.csv', index=False)